## Scrape Store Hours on Intermix Website

#### To practice scraping HTML and formatting the output, I decided to scrape store location information from Intermix, a womenswear retailer that I like.

#### Using BeautifulSoup, I extract informationa about all 30 stores and format it in a data frame that would make further analysis much easier.

In [48]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url2 = "https://www.intermixonline.com/storeresults?dwfrm_storelocator_distanceUnit=mi&dwfrm_storelocator_findall=Search"
page2 = requests.get(url2)
soup2 = BeautifulSoup(page2.content)

In [2]:
#hwo to view what our request heder is
page2.request.headers

{'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

#### Now let's try to get Intermix working!

In [3]:
#INTERMIX ADDRESSES
addresses = soup2.find_all("div", attrs = {"class": "column small-12 medium-4 large-4 align-self-middle store-address"})

In [9]:
# convert string from bs4 element to text
convert_to_string = []
for x in addresses:
  xx = str(x).replace('<div class="column small-12 medium-4 large-4 align-self-middle store-address">', '')
  xx = xx.replace('</div>', '')
  xx = xx.replace('\n', '')
  convert_to_string.append(str(xx))

# view the new list
convert_to_string

['Scottsdale Quarter<br/>15257 N. Scottsdale Rd Suite F1-100<br/>Scottsdale, AZ 85254<br/>United States',
 '400 N Beverly Drive<br/>Beverly Hills, CA 90210<br/>United States',
 'Brentwood Country Mart<br/>225 26th St.,  Space 52<br/>Santa Monica, CA 90402<br/>United States',
 '2223 Fillmore Street<br/>San Francisco, CA 94115<br/>United States',
 'Malibu Lumber Yard<br/>3939 Cross Creek Rd., Suite B110<br/>Malibu, CA 90265<br/>United States',
 'Marin Country Mart<br/>2409 Larkspur Landing Circle<br/>Larkspur, CA 94939<br/>United States',
 '110 N Robertson Blvd<br/>Los Angeles, CA 90048<br/>United States',
 'South Coast Plaza<br/>3333 Bristol Street<br/>Costa Mesa, CA 92626<br/>United States',
 '428 E Hyman Ave<br/>Aspen, CO 81611<br/>United States',
 '325 Greenwich Ave<br/>Greenwich, CT 06830<br/>United States',
 '51 Main Street<br/>Westport, CT 06880<br/>United States',
 '3300 M Street NW<br/>Washington, DC 20007<br/>United States',
 'Bal Harbour Shops<br/>9700 Collins Avenue, Suite #2

In [46]:
# parse list and create new line based on line breaks
line_breaks = []

for x in convert_to_string:
  for y in range(0, len(x) - 5):
    if x[y:(y + 5)] == '<br/>':
      line_breaks.append(y)

# now separate the list based on the line breaks

clean_list = []
count_1 = 0
for x in convert_to_string:
  clean_line = x[0 : line_breaks[count_1]]
  clean_list.append(clean_line)
  clean_line = x[line_breaks[count_1] + 5: line_breaks[count_1 + 1]]
  clean_list.append(clean_line)

  if line_breaks[count_1 + 1] < line_breaks[count_1 + 2]:
    clean_line = x[line_breaks[count_1 + 1] + 5: line_breaks[count_1 + 2]]
    clean_list.append(clean_line)
    clean_line = x[line_breaks[count_1 + 2] + 5: len(x)]
    clean_list.append(clean_line)
    count_1 += 3
  else:
    clean_line = x[line_breaks[count_1 + 1] + 5: len(x)]
    clean_list.append(clean_line)
    count_1 += 2

clean_list

['Scottsdale Quarter',
 '15257 N. Scottsdale Rd Suite F1-100',
 'Scottsdale, AZ 85254',
 'United States',
 '400 N Beverly Drive',
 'Beverly Hills, CA 90210',
 'United States',
 'Brentwood Country Mart',
 '225 26th St.,  Space 52',
 'Santa Monica, CA 90402',
 'United States',
 '2223 Fillmore Street',
 'San Francisco, CA 94115',
 'United States',
 'Malibu Lumber Yard',
 '3939 Cross Creek Rd., Suite B110',
 'Malibu, CA 90265',
 'United States',
 'Marin Country Mart',
 '2409 Larkspur Landing Circle',
 'Larkspur, CA 94939',
 'United States',
 '110 N Robertson Blvd',
 'Los Angeles, CA 90048',
 'United States',
 'South Coast Plaza',
 '3333 Bristol Street',
 'Costa Mesa, CA 92626',
 'United States',
 '428 E Hyman Ave',
 'Aspen, CO 81611',
 'United States',
 '325 Greenwich Ave',
 'Greenwich, CT 06830',
 'United States',
 '51 Main Street',
 'Westport, CT 06880',
 'United States',
 '3300 M Street NW',
 'Washington, DC 20007',
 'United States',
 'Bal Harbour Shops',
 '9700 Collins Avenue, Suite #2

In [200]:
# convert list to an organized table
hours_df = pd.DataFrame(columns = ('Mall Name', 'Address', 'City & Zip', 'Country'))

In [201]:
i = 0
j = 0
k = 0

while k <= len(clean_list) - 2:

  k = i
  hours_df.loc[j, 'Mall Name'] = clean_list[k]
  
  if clean_list[k] == "United States":
    j += 1

  k = i + 1
  hours_df.loc[j, 'Address'] = clean_list[k]
  
  if clean_list[k] == "United States":
    j += 1

  k = i + 2
  hours_df.loc[j, 'City & Zip'] = clean_list[k]
  
  if clean_list[k] == "United States":
    hours_df.loc[j, 'Mall Name'] = None
    hours_df.loc[j, 'Address'] = clean_list[k - 2]
    hours_df.loc[j, 'City & Zip'] = clean_list[k - 1]
    hours_df.loc[j, 'Country'] = clean_list[k]
    j += 1
    i += 3
  else:
    k = i + 3
    hours_df.loc[j, 'Country'] = clean_list[k]
    j += 1
    i += 4
  
hours_df

,Mall Name,Address,City & Zip,Country
0,Scottsdale Quarter,15257 N. Scottsdale Rd Suite F1-100,"Scottsdale, AZ 85254",United States
1,None,400 N Beverly Drive,"Beverly Hills, CA 90210",United States
2,Brentwood Country Mart,"225 26th St., Space 52","Santa Monica, CA 90402",United States
3,None,2223 Fillmore Street,"San Francisco, CA 94115",United States
4,Malibu Lumber Yard,"3939 Cross Creek Rd., Suite B110","Malibu, CA 90265",United States
5,Marin Country Mart,2409 Larkspur Landing Circle,"Larkspur, CA 94939",United States
6,None,110 N Robertson Blvd,"Los Angeles, CA 90048",United States
7,South Coast Plaza,3333 Bristol Street,"Costa Mesa, CA 92626",United States
8,None,428 E Hyman Ave,"Aspen, CO 81611",United States
9,None,325 Greenwich Ave,"Greenwich, CT 06830",United States


In [209]:
# create empty columns
hours_df['Zip'] = None
hours_df['City'] = None
hours_df['State'] = None

# isolate specific strings
for i in range(0, 30):
  hours_df['Zip'][i] = hours_df['City & Zip'][i][-5:] # isolate zip code

  last_city_position = hours_df['City & Zip'][i].find(',')
  hours_df['City'][i] = hours_df['City & Zip'][i][0:last_city_position] # isolate city
  
  # isolate state
  hours_df['State'][i] = hours_df['City & Zip'][i][last_city_position + 2:last_city_position + 4]

# drop the City & Zip column now that we've isolated the contents into three separate columns
hours_df.drop('City & Zip', axis = 1, inplace = True)

In [211]:
hours_df = hours_df[['Mall Name', 'Address', 'City', 'State', 'Zip', 'Country']]

# preview final data frame
hours_df

,Mall Name,Address,City,State,Zip,Country
0,Scottsdale Quarter,15257 N. Scottsdale Rd Suite F1-100,Scottsdale,AZ,85254,United States
1,None,400 N Beverly Drive,Beverly Hills,CA,90210,United States
2,Brentwood Country Mart,"225 26th St., Space 52",Santa Monica,CA,90402,United States
3,None,2223 Fillmore Street,San Francisco,CA,94115,United States
4,Malibu Lumber Yard,"3939 Cross Creek Rd., Suite B110",Malibu,CA,90265,United States
5,Marin Country Mart,2409 Larkspur Landing Circle,Larkspur,CA,94939,United States
6,None,110 N Robertson Blvd,Los Angeles,CA,90048,United States
7,South Coast Plaza,3333 Bristol Street,Costa Mesa,CA,92626,United States
8,None,428 E Hyman Ave,Aspen,CO,81611,United States
9,None,325 Greenwich Ave,Greenwich,CT,06830,United States
